In [3]:
import pandas as pd
import numpy as np
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM, Dropout
from keras.layers import Dense

Using TensorFlow backend.


# Reading Data

In [4]:
df = pd.read_excel('PrivatizedDataforParticipants.xlsx')
new_header = df.iloc[1] #grab the first row for the header
df = df[2:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df2 = df.drop(['Generic Group','Generic Brand','Generic Product Category','Generic Sub-Variable','Units'],axis=1)
df2 = df2.rename(columns={'Generic LookupKey': 'Generic_LookupKey', 'Generic Variable': 'Generic_Variable'})
df2.reset_index(inplace = True, drop = True)

df2 = df2[df2.Generic_Variable == 'Closing Base']
df2.head()

1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
2,Panther,Volume,Closing Base,Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,-2.34673,-2.19826,-2.04405,-1.85444,-1.69442,-1.51188,...,0.95662,1.00444,1.04301,1.08287,x,x,x,x,x,x
5,Leopard,Volume,Closing Base,Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,-2.04374,-2.03103,-1.94628,-1.7605,-1.58287,-1.46612,...,0.890681,0.997703,1.088,1.15985,x,x,x,x,x,x
8,Lion,Volume,Closing Base,Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,NaN,NaN,NaN,NaN,NaN,NaN,...,0.855154,0.899006,0.948791,0.997987,x,x,x,x,x,x


## Helper Functions

In [5]:
# split a univariate sequence
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse})

## Panther

In [6]:
df3 = df2.iloc[0:1,:]
df3.head()

1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
2,Panther,Volume,Closing Base,Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,-2.34673,-2.19826,-2.04405,-1.85444,-1.69442,-1.51188,...,0.95662,1.00444,1.04301,1.08287,x,x,x,x,x,x


In [7]:
df3 = df3.iloc[:, 10:46]
df3 = df3.melt(var_name="Month", value_name="Value")

In [8]:
raw_seq = np.asarray(df3.Value)
# choose a number of time steps
n_steps = 5
n_neurons = 40
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(n_neurons, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mape')
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [9]:
# vary par = n_steps , lstm_units , droput,dense , activation , loss , optimizer

In [10]:
fc = []
for arr in X:
    x_input = []
    for i in arr:
        x_input.append(i[0])
    x_input = np.asarray(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    fc.append(yhat[0][0])
actual = np.asarray(df3.Value)[n_steps:]
print(fc,actual,len(fc),len(actual))

[-0.8801699, -0.62629503, -0.4627735, -0.3519528, -0.26519683, -0.18351354, -0.09574138, 0.0120506985, 0.088351525, 0.14688838, 0.19749975, 0.24895555, 0.29795337, 0.333655, 0.3647571, 0.39974973, 0.42879048, 0.46266848, 0.49043703, 0.5246698, 0.56986713, 0.6146184, 0.66118574, 0.71093446, 0.7647387, 0.82658285, 0.8642761, 0.9187539, 0.97173685, 1.0341908, 1.0988852] [-0.7153560205695702 -0.6456841998306309 -0.5488377497621234
 -0.43224058265378257 -0.32551468150070045 -0.22147350276331038
 -0.08430710568352238 0.03023559396646883 0.09672339940143161
 0.13399029317779354 0.20233612296751965 0.23464282837830389
 0.2986914652770594 0.3260625377102138 0.34800686940677167
 0.409280747467357 0.461084430909174 0.5085745512013885 0.553830131190298
 0.5906877685008536 0.6876160698625223 0.6984368108184525
 0.748963614086953 0.7806400645403948 0.8267960087540793
 0.8671486962826237 0.9185431232586343 0.956620344837294
 1.0044378703021535 1.0430061996397808 1.0828677794093575] 31 31


In [11]:
forecast_accuracy(fc,actual)

{'mape': 0.10394238451432304,
 'me': -0.02158888570755408,
 'mae': 0.04427210572938097,
 'mpe': -0.06462053387294506,
 'rmse': 0.057945541413075806}

In [12]:
df4 = df2.iloc[0:1,:]
for i in range(0,6):
    x_input = df4.iloc[:,i+46-n_steps:i+46].values.tolist()[0]
    x_input = np.asarray(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    df4.iloc[:,i+46] = yhat[0][0]
df4

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
2,Panther,Volume,Closing Base,Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,-2.34673,-2.19826,-2.04405,-1.85444,-1.69442,-1.51188,...,0.95662,1.00444,1.04301,1.08287,1.169113,1.24054,1.32653,1.42781,1.56382,1.75589


## Leopard

In [15]:
df3 = df2.iloc[1:2,:]
df3.head()

1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
5,Leopard,Volume,Closing Base,Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,-2.04374,-2.03103,-1.94628,-1.7605,-1.58287,-1.46612,...,0.890681,0.997703,1.088,1.15985,x,x,x,x,x,x


In [16]:
df3 = df3.iloc[:, 10:46]
df3 = df3.melt(var_name="Month", value_name="Value")

In [17]:
raw_seq = np.asarray(df3.Value)
# choose a number of time steps
n_steps = 3
n_neurons = 80
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(n_neurons, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mape')
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [18]:
fc = []
for arr in X:
    x_input = []
    for i in arr:
        x_input.append(i[0])
    x_input = np.asarray(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    fc.append(yhat[0][0])
actual = np.asarray(df3.Value)[n_steps:]
print(fc,actual,len(fc),len(actual))

[-1.6381334, -1.1427324, -0.788749, -0.60256374, -0.467038, -0.3498462, -0.26469895, -0.19873089, -0.13910018, -0.09521483, -0.05777622, -0.02120275, 0.0008437021, 0.020920832, 0.08203579, 0.14614183, 0.20809372, 0.26336014, 0.31081071, 0.34703594, 0.3695767, 0.39109236, 0.41600224, 0.45892334, 0.5177576, 0.629632, 0.6813466, 0.7468689, 0.8147708, 0.9211941, 0.9999642, 1.0934849, 1.2023042] [-0.836258534676948 -0.7392399795443133 -0.6306858216933975
 -0.5421696032256311 -0.47684714639860537 -0.40124528794612585
 -0.3412080674242886 -0.2912204556450706 -0.23466076741679226
 -0.19983948590463713 -0.13900565005564092 -0.06416976787927564
 -0.001083961778503588 0.06561725234552815 0.12635917104139022
 0.1803911375584918 0.22494031777729812 0.24893069474519744
 0.2688154388731242 0.28755121858687965 0.32482362418260047
 0.3444479363766484 0.4793823171769399 0.5190292492286149
 0.5772281266877781 0.6113447100259082 0.7103088448998733
 0.7662097935306935 0.8326046171442795 0.8906809383992641


In [19]:
forecast_accuracy(fc,actual)

{'mape': 0.2744786435443855,
 'me': -0.028793981268160313,
 'mae': 0.08108685094709153,
 'mpe': -0.11271748615520091,
 'rmse': 0.16615272278373897}

In [20]:
df4 = df2.iloc[1:2,:]
for i in range(0,6):
    x_input = df4.iloc[:,i+46-n_steps:i+46].values.tolist()[0]
    x_input = np.asarray(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    df4.iloc[:,i+46] = yhat[0][0]
df4

1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
5,Leopard,Volume,Closing Base,Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,-2.04374,-2.03103,-1.94628,-1.7605,-1.58287,-1.46612,...,0.890681,0.997703,1.088,1.15985,1.351402,1.50265,1.69924,2.03313,2.41174,3.01655


## Lion

In [21]:
df3 = df2.iloc[2:3,:]
df3.head()

1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
8,Lion,Volume,Closing Base,Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,NaN,NaN,NaN,NaN,NaN,NaN,...,0.855154,0.899006,0.948791,0.997987,x,x,x,x,x,x


In [22]:
df3 = df3.iloc[:, 10:46]
df3 = df3.melt(var_name="Month", value_name="Value")

In [23]:
raw_seq = np.asarray(df3.Value)
# choose a number of time steps
n_steps = 2
n_neurons = 120
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(n_neurons, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mape')
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [24]:
fc = []
for arr in X:
    x_input = []
    for i in arr:
        x_input.append(i[0])
    x_input = np.asarray(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    fc.append(yhat[0][0])
actual = np.asarray(df3.Value)[n_steps:]
print(fc,actual,len(fc),len(actual))

[-1.8805493, -1.7035421, -1.5077611, -1.3156669, -1.1563406, -1.0028539, -0.906483, -0.7645646, -0.5959804, -0.48357308, -0.37019596, -0.24303831, -0.15711889, -0.050549608, 0.03523632, 0.116853036, 0.17820676, 0.23734802, 0.28148243, 0.3076752, 0.33806077, 0.37175682, 0.40817428, 0.45031753, 0.48416108, 0.54752, 0.5831759, 0.6391577, 0.69371736, 0.75807244, 0.80148566, 0.85547155, 0.91095525, 0.9727581] [-1.8798407668040886 -1.730280973331413 -1.60987520306263
 -1.4544984274809205 -1.3732680159306039 -1.2469185570117978
 -1.0338557489507936 -0.8865305981101037 -0.7443888966735036
 -0.533975407196812 -0.4208379848702089 -0.29623937997233984
 -0.1500776255534561 -0.04295300453559756 0.03718312325163241
 0.1291951095624562 0.2028346474298945 0.23917638714079184
 0.27991829945693897 0.325279241130953 0.3677374511386763
 0.4267020678043129 0.45282665280092627 0.5348173035971757
 0.5653420611990388 0.6236155512774635 0.6706006404314256 0.73480630038342
 0.7689594725681461 0.8128575296325613

In [25]:
forecast_accuracy(fc,actual)

{'mape': 0.09638601951049866,
 'me': 0.010709363789858643,
 'mae': 0.06471612938845706,
 'mpe': -0.08287186270489041,
 'rmse': 0.08694156546557606}

In [26]:
df4 = df2.iloc[2:3,:]
for i in range(0,6):
    x_input = df4.iloc[:,i+46-n_steps:i+46].values.tolist()[0]
    x_input = np.asarray(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    df4.iloc[:,i+46] = yhat[0][0]
df4

1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
8,Lion,Volume,Closing Base,Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,NaN,NaN,NaN,NaN,NaN,NaN,...,0.855154,0.899006,0.948791,0.997987,1.044087,1.11753,1.20227,1.33357,1.5161,1.82808
